<a href="https://colab.research.google.com/github/charlesdgburns/NM_TinyRNN/blob/main/notebooks/nm_tinyrnn_dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Let's use this as a learning notebook about different RNN architectures.
We want to start from scratch and get an idea of how different architectures work before fitting to some data.


We the fit these to sequential behavioural decision making later.


In [ ]:
## setup on google colab:

!git clone https://github.com/charlesdgburns/NM_TinyRNN.git

Cloning into 'NM_TinyRNN'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 104 (delta 37), reused 55 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (104/104), 212.00 KiB | 4.93 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [115]:
# setup

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

import os
CODE_DIR = Path(Path(r'.'))
os.chdir(CODE_DIR)


In [116]:
# Train some models using the standard code

from NM_TinyRNN.code.models import training
from NM_TinyRNN.code.models import datasets
from NM_TinyRNN.code.models import rnns
from importlib import reload
reload(training)
reload(datasets)
SAVE_PATH = CODE_DIR/'NM_TinyRNN/data/rnns'
SPARSITY_LAMBDA = 0.0001
DATA_PATH = './NM_TinyRNN/data/AB_behaviour/WS16'
SEQUENCE_LENGTH = 150+1 # Define your desired sequence length
dataset = datasets.AB_Dataset(DATA_PATH, SEQUENCE_LENGTH)
model = rnns.TinyRNN(input_size=3, # past choice, past outcome, past forced choice
                     hidden_size=1, # hidden unit
                     out_size=2, # one-hot code for choice A, choice B
                     rnn_type='GRU', # GRU, LSTM, NMRNN
                sparsity_lambda = SPARSITY_LAMBDA) # Example sizes
trainer = training.Trainer(SAVE_PATH/'GRU')
trainer.fit(model,dataset)

FileNotFoundError: [Errno 2] No such file or directory: 'NM_TinyRNN/data/AB_behaviour/WS16'

In [103]:
# Extract trial-wise data for visualisation
trials_results = pd.read_csv('./NM_TinyRNN/data/RNNs/GRU/1_unit_GRU_trials_data.htsv', sep='\t')
trials_inputs = dataset.inputs.reshape(-1, 3).numpy().astype(int)
trials_results.loc[:, ["forced_choice","outcome", "choice"]] = trials_inputs # "forced_choice","outcome","choice"
trials_results

FileNotFoundError: [Errno 2] No such file or directory: './NM_TinyRNN/data/RNNs/GRU/1_unit_GRU_trials_data.htsv'

In [108]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_phase_portrait_direct(trials_results, s=6, alpha=0.7):
    """
    Plots the phase portrait directly from existing DataFrame columns:
    x = logit_value, y = logit_change.
    """
    # --- 1. Prepare data by dropping rows with no logit_change ---
    df = trials_results.dropna(subset=['logit_value', 'logit_change']).copy()

    # --- 2. Directly use columns to create trial types for coloring ---
    # A1,R=0 -> 0; A1,R=1 -> 1; A2,R=0 -> 2; A2,R=1 -> 3
    df['trial_type'] = df['choice'] * 2 + df['outcome']

    colors = ["cornflowerblue", "mediumblue", "tomato", "firebrick"]

    # --- 3. Plotting ---
    plt.figure(figsize=(3.5, 3.5))

    for tt in range(4):
        mask = (df['trial_type'] == tt)
        label = f'A{tt//2 + 1}, R={tt%2}' # e.g., A1, R=0
        plt.scatter(df.loc[mask, 'logit_value'], df.loc[mask, 'logit_change'],
                    c=colors[tt], s=s, alpha=alpha, label=label)

    # --- 4. Formatting ---

    # Reference lines
    plt.axhline(0, color='k', alpha=0.3, linewidth=1, linestyle='--')
    plt.axvline(0, color='k', alpha=0.3, linewidth=1, linestyle='--')

    plt.xlabel("logit")
    plt.ylabel("logit change")
    plt.title("GRU")
    plt.legend()
    plt.tight_layout()
    plt.show()

# --- Usage ---
plot_phase_portrait_direct(trials_results)

KeyError: 'choice'

In [107]:
# Extract trial-wise data for visualisation
trials_results = pd.read_csv('.data/rnns/GRU/2_unit_GRU_trials_data.htsv', sep='\t')
trials_inputs = dataset.inputs.reshape(-1, 3).numpy().astype(int)
trials_results.loc[:, ["forced_choice","outcome", "choice"]] = trials_inputs # "forced_choice","outcome","choice"
trials_results

FileNotFoundError: [Errno 2] No such file or directory: '.data/rnns/GRU/2_unit_GRU_trials_data.htsv'